# Testing the trained model on unseen data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils
import torch.utils.data 
import torch.utils.data.dataloader
import torchvision.models.resnet
from torchvision import datasets, transforms, models

from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score
from itertools import groupby

In [9]:
print(f"Is MPS built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Is MPS built? True
Is MPS available? True
Using device: mps


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Absoluter Pfad basierend auf dem aktuellen Arbeitsverzeichnis
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Relative path to the folder containing the images
relative_folderpath = os.path.join(parent_directory, 'data/test/pl')

test_dataset = datasets.ImageFolder(relative_folderpath, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

# Modell initialisieren (z.B. ResNet)
model = models.resnet50(weights=None)
num_ftrs = model.fc.in_features

# Gewichte laden
#checkpoint = torch.load('model/resNet_complete.pth')
checkpoint = torch.load('model/AlexNet_complete.pth')

# Passen Sie die letzte Schicht an die Anzahl der Klassen an
model.fc = torch.nn.Linear(num_ftrs, len(test_dataset.classes))

# Laden Sie die Gewichte, aber ignorieren Sie die letzte Schicht
pretrained_dict = checkpoint['model_state_dict']
model_dict = model.state_dict()

# Filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and model_dict[k].shape == v.shape}

# Overwrite entries in the existing state dict
model_dict.update(pretrained_dict)

# Load the new state dict
model.load_state_dict(model_dict)
model.eval()  # Setzen Sie das Modell in den Evaluierungsmodus

# Evaluierung
correct = 0
total = 0
true_labels = []
predicted_labels = []

with torch.no_grad():  # Kein Gradient Tracking für Evaluierung nötig
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels)
        predicted_labels.extend(predicted)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Genauigkeit auf den Testdaten: {accuracy:.2f}%')

f1 = f1_score(true_labels, predicted_labels, average='weighted')
print(f'F1 Score: {f1}')

precision = precision_score(true_labels, predicted_labels, average='weighted')
print(f'Precision: {precision}')

recall = recall_score(true_labels, predicted_labels, average='weighted')
print(f'Recall: {recall}')


/var/folders/4_/cp0xvtt57bs8hr0q5bv1vnw80000gn/T/ipykernel_8160/472820281.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('model/AlexNet_complet

Genauigkeit auf den Testdaten: 41.71%


In [11]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Create a DataFrame from the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
cm_df = pd.DataFrame(cm, index=['Actual Corrected', 'Actual Original', 'Actual Smoothed'], columns=['Predicted Corrected', 'Predicted Original', 'Predicted Smoothed'])

# Display the DataFrame
print(cm_df)

                  Predicted Corrected  Predicted Original  Predicted Smoothed
Actual Corrected                   78                   0                   0
Actual Original                    55                   0                   0
Actual Smoothed                    54                   0                   0
